In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from numpy.random import randn

import matplotlib.pyplot as plt
import seaborn as sns
import dataprep as dp
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet

import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import cross_val_score,KFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, RANSACRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [5]:
train_df = pd.read_csv('train.csv')
train_df.drop(['id'], axis=1, inplace=True)
column_names = list(train_df.columns.values.tolist())

In [6]:
train_df.drop(index=[934, 1035], inplace=True)

In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

train = train_df.copy()

train = IterativeImputer(random_state=2021).fit_transform(train)

train_imp = pd.DataFrame(train)
train_imp.columns = column_names

train_imp.isna().sum()

hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [8]:
columns=['hour','hour_bef_temperature','hour_bef_humidity','hour_bef_ozone','hour_bef_windspeed','count']
train_fin=train_imp[columns]

In [9]:
from sklearn.ensemble import IsolationForest 
model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1, max_features=2, contamination=0.01, random_state=2021)
model.fit(train_fin.to_numpy())

IsolationForest(contamination=0.01, max_features=2, n_jobs=-1,
                random_state=2021)

In [10]:
score = model.decision_function(train_fin.to_numpy())
anomaly = model.predict(train_fin.to_numpy())
train_fin['scores']= score
train_fin['anomaly']= anomaly
anomaly_data = train_fin.loc[train_fin['anomaly']==-1] # 이상값은 -1으로 나타낸다.
anomaly_data

NumExpr defaulting to 8 threads.


,hour,hour_bef_temperature,hour_bef_humidity,hour_bef_ozone,hour_bef_windspeed,count,scores,anomaly
4,18.0,29.5,7.0,0.057,4.8,431.0,-0.057651,-1
117,15.0,28.4,43.0,0.099,5.1,97.0,-0.009766,-1
245,16.0,21.1,21.0,0.075,5.6,382.0,-0.008752,-1
292,16.0,29.0,41.0,0.106,4.1,126.0,-0.001004,-1
339,19.0,28.7,8.0,0.052,3.3,317.0,-0.003443,-1
358,17.0,28.1,19.0,0.088,2.0,248.0,-0.011317,-1
362,15.0,29.6,22.0,0.099,2.5,227.0,-0.017209,-1
448,18.0,29.3,32.0,0.082,2.6,380.0,-0.014948,-1
641,16.0,30.0,16.0,0.100,3.2,304.0,-0.046998,-1
784,8.0,3.2,83.0,0.004,0.4,21.0,-0.003223,-1


In [11]:
outlier_index=train_fin.index[train_fin['anomaly']==-1].tolist()
for i in outlier_index:
    train_fin=train_fin.drop(index=i,axis=0)
    
train_fin=train_fin.drop(train_fin[['anomaly']],axis=1)
train_fin=train_fin.drop(train_fin[['scores']],axis=1)

In [12]:
columns_fin=['hour','hour_bef_temperature','hour_bef_humidity','hour_bef_ozone','hour_bef_windspeed']
X=train_fin[columns_fin]
y=train_fin['count']

In [13]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3, random_state=2021)

In [14]:
from sklearn.ensemble import StackingRegressor

best_reg = ExtraTreesRegressor(n_estimators=500, max_depth=15, random_state=2021)

estimators= [('rf_reg',RandomForestRegressor(max_depth=15, n_jobs=-1, n_estimators=500, random_state=2021)),
('aba_reg',AdaBoostRegressor(n_estimators=500,learning_rate=0.3,random_state=2021)),
('dt_reg',DecisionTreeRegressor(max_depth=15,random_state=2021))]

reg= StackingRegressor(estimators=estimators, final_estimator=best_reg)
reg.fit(X_train, y_train)

StackingRegressor(estimators=[('rf_reg',
                               RandomForestRegressor(max_depth=15,
                                                     n_estimators=500,
                                                     n_jobs=-1,
                                                     random_state=2021)),
                              ('aba_reg',
                               AdaBoostRegressor(learning_rate=0.3,
                                                 n_estimators=500,
                                                 random_state=2021)),
                              ('dt_reg',
                               DecisionTreeRegressor(max_depth=15,
                                                     random_state=2021))],
                  final_estimator=ExtraTreesRegressor(max_depth=15,
                                                      n_estimators=500,
                                                      random_state=2021))

In [17]:
y_pred=reg.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test,y_pred))
print(rmse)

41.7163424216256


In [21]:
test_df=pd.read_csv('test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


In [22]:
test = test_df.copy()

test = IterativeImputer(random_state=2021).fit_transform(test)

test_imp = pd.DataFrame(test)
test_imp.columns = column_names

test_imp.isna().sum()

hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [26]:
columns_fin=['hour','hour_bef_temperature','hour_bef_humidity','hour_bef_ozone','hour_bef_windspeed']
X_test_df=test_imp[columns_fin]
X_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hour                  715 non-null    float64
 1   hour_bef_temperature  715 non-null    float64
 2   hour_bef_humidity     715 non-null    float64
 3   hour_bef_ozone        715 non-null    float64
 4   hour_bef_windspeed    715 non-null    float64
dtypes: float64(5)
memory usage: 28.1 KB


In [28]:
test_pred=reg.predict(X_test_df)

In [31]:
submission=pd.read_csv('submission.csv')
submission['count']=test_pred
submission

,id,count
0,0,39.126695
1,1,26.123071
2,2,33.999256
3,4,9.477891
4,5,33.779060
...,...,...
710,2148,114.279322
711,2149,114.279322
712,2165,110.581322
713,2166,139.632695


In [33]:
submission.to_csv('model_if_stacking.csv',index=False)